In [1]:
import numpy as np
import pandas as pd
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio

plt.style.use('ggplot')
sns.set(font_scale=1.5)

pio.renderers.default = "svg"

In [2]:
df = pd.read_csv('data.csv', index_col='date')

# Defining Target Values and Tran Test Split

Как я понимаю нашу задачу: по историческим данным предсказать Revenue и EBITDA на один следующий временной период (t + 1)

In [3]:
df['target_revenue'] = df.groupby('Company')[['Revenue']].shift(-1)
df['target_ebitda'] = df.groupby('Company')[['EBITDA']].shift(-1)

df.dropna(axis=0, inplace=True)

In [4]:
df_train = df.loc[df.index.values[:37], :].reset_index().sort_values(by=['Company', 'date']).set_index('date')
df_test = df.loc[df.index.values[37:41], :].reset_index().sort_values(by=['Company', 'date']).set_index('date')

# Exploratory Data Analysis

Уберём строки, которые состоят из одних нулей или практически из одних нулей (не считая таргетных переменных, так как они со сдвигом, и столбца Company)

In [5]:
df_train = df_train.loc[
    ~((df_train.drop(columns=['target_revenue', 'target_ebitda', 'Company'])==0).sum(axis=1) / 13 > 0.5)
]

Как мы можем увидеть ниже, для большинства компаний есть данные по 37 кварталам, однако имеется ряд компаний с очень короткой историей. Например MNRL (5 кварталов) и BRY (10 кварталов).

In [6]:
df_train.groupby('Company')['Revenue'].count()

Company
AMPY    34
APA     37
AR      34
BATL    37
BCEI    37
BRY     10
BSM     22
CDEV    17
CHK     37
CLR     37
CNX     37
COG     37
COP     37
CPE     37
CRC     23
CRK     37
DEN     37
DVN     37
EGY     37
EOG     37
EQT     37
ESTE    37
FANG    35
GDP     37
HES     37
LONE    17
LPI     35
MCF     37
MGY     12
MNRL     5
MRO     37
MTDR    37
MUR     37
NOG     37
OAS     37
OVV     37
PDCE    37
PVAC    37
PXD     37
REI     30
RRC     37
SBOW    36
SD      37
SM      37
SWN     37
TALO    37
TPL     37
VNOM    25
WLL     37
WTI     37
XEC     37
XOG     17
Name: Revenue, dtype: int64

Поищем пропуски

In [7]:
df_train.groupby('Company').sum() == 0

,Revenue,Gross Profit,Operating Income,Pretax Income,Income (Loss) from Cont Ops,Net Income,Net Income Avail to Common,Diluted EPS,EBITDA,Total Equity,Cash from Operations,Cash from Investing Activities,Cash from Financing Activities,target_revenue,target_ebitda
Company,,,,,,,,,,,,,,,
AMPY,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
APA,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
AR,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
BATL,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
BCEI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
BRY,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
BSM,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
CDEV,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
CHK,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


*У VNOM в EBITDA одни нули, у LONE в Diluted EPS одни нули и у MGY в Gross Profit одни нули.*

Если честно, то пока непонятно как обрабатывать эти пропуски, так как не до конца ясна сама задача. Если нам надо предсказывать EBITDA, то VNOM можно убирать из анализа, или же оставлять, но тогда прогнозировать для него только Revenue. Что касается двух остальных кейсов, то необходимо сначала посмотреть как ведут себя все признаки. Возможно они все очень сильно коррелированы, и нам в любом случае придётся убирать Diluted EPS и Gross Profit.

Совокупная таблица со всеми статистиками по нашим данным представлена ниже. В целом, у нас много разных компаний с различными значениями показателей, поэтому сразу что-то, что выделяется, выделить тяжело.

In [8]:
df_train.groupby('Company').agg(['mean', 'std', 'min', 'max'])

Revenue                                         Gross Profit  \
                 mean          std          min           max         mean   
Company                                                                      
AMPY        88.108470    39.938381    48.410000    166.518000    35.257206   
APA       2584.189191  1242.134938  1084.000000   4496.000000   949.513527   
AR         657.956852   349.468652    92.034000   1242.779008   244.179495   
BATL       138.883650    99.732188    24.197000    311.999008    57.707649   
BCEI        74.981866    34.539640    21.436696    155.376992    33.311001   
BRY        163.760198    87.953166    65.982000    339.264992    85.487398   
BSM        116.492500    41.166258    64.803000    246.047008    72.192864   
CDEV       152.086177    89.681521    23.750000    252.944000    47.418296   
CHK       3076.297306  1004.789061  1995.000064   5190.000128  1979.486506   
CLR        826.168758   343.652588   -36.210000   1645.328000   319.943761   
CNX        719.725271   386.420774   238.336992   1563.006976   167.229895   
COG        406.030486   124.863689   209.008992    716.300992   140.384459   
COP      11476.513474  8053.475689  5123.999744  54174.998528  2192.621513   
CPE         74.754730    62.006459    23.163000    289.919008    31.831622   
CRC        625.739130   159.630032   322.000000   1092.000000   196.913043   
CRK        104.976243    54.041631    36.163000    289.248000    14.632702   
DEN        442.098136   154.563538   234.242000    672.880000   180.791756   
DVN       2699.459463   865.829524  1079.000064   4800.999936   610.297292   
EGY         30.481054    15.175073    10.976000     67.767000    17.731892   
EOG       3195.119042   922.971683  1357.453952   4625.866752  1668.185093   
EQT        649.246891   268.981907   338.593984   1231.453056   193.232432   
ESTE        17.908973    18.130172     1.764000     68.994000     7.461189   
FANG       290.509447   331.472659    10.963589   1116.000000   132.583322   
GDP         36.000027    19.725702     6.245000     65.364000     7.729622   
HES       2969.729721  2603.092168   973.000000  10085.000192  1581.972957   
LONE        34.685321    13.078476    18.023000     55.709000     9.479984   
LPI        200.208457    43.910047   133.440000    345.776000    80.727116   
MCF         34.507855    18.066349    12.547000     78.555000     8.724834   
MGY        184.291834    78.498772     0.000000    255.055008     0.000000   
MNRL        26.759000     6.271413    18.265000     33.614000    18.499600   
MRO       2139.783789  1153.604310   616.000000   4124.999936  1001.216232   
MTDR       113.257987    78.612097    15.548328    312.687008    43.635248   
MUR       1898.810875  2328.875729   366.827008   7384.573952   342.635110   
NOG         99.505303   100.544182    -6.934000    506.784992    50.698113   
OAS        303.740270   139.073367    58.232000    589.177984    86.138135   
OVV       1510.378385   453.590087   807.000000   2643.000064   429.297306   
PDCE       186.485484    95.417978    61.011000    368.155008    65.133917   
PVAC        89.522298    30.622851    32.929000    134.319008    26.408946   
PXD       1303.016251   615.339846   507.105984   2654.000128   377.641629   
REI         17.378730    15.784098     0.328003     52.231184     6.459360   
RRC        511.789297   163.290112   251.987008    880.689024   152.918272   
SBOW        95.076610    45.876961    29.163000    157.904992    27.796137   
SD         266.850972   252.497870    40.329000   1377.181952   102.148054   
SM         417.156893   114.841018   212.096992    674.979968    95.980407   
SWN        802.837923   133.716594   545.000000   1076.000000   377.193789   
TALO       187.380485    60.860150    80.877000    286.809984    58.496539   
TPL         36.808500    41.293215     5.715768    191.324000    18.095562   
VNOM        42.800320    26.313233    14.194000     92.711000    28.094720   
WLL        505.161916   146.003563  

## Data Vizualization

In [ ]:
px.line(df_train, y="Revenue", color='Company', title='Revenue Dynamics by Company')

In [ ]:
px.line(df_train, y="EBITDA", color='Company', title='EBITDA Dynamics by Company')

На первый взгляд кажется, что EBITDA по сравнению с Revenue более динамична, однако дело в масштабе, и в целом, EBITDA очень сильно коррелирует с Revenue.

Посчитаем эту корреляцию

Вспомним, что у VNOM EBITDA везде равна нулю, поэтому уберем её, чтобы корреляция посчиталась.

In [ ]:
df_train[~(df_train['Company'] == 'VNOM')].groupby('Company')[
    ['Revenue', 'EBITDA']].corr()['Revenue'].values[1::2].mean()

Грубая средняя оценка корреляции без учёта количество наблюдений для каждой компаним составила 0.8599. Что говорит нам о том, что в целом, необходимости в построении двух моделей, чтобы одна предсказывала Revenue, а вторая EBITDA, у нас нет.
Можно, например, прогнозировать только Revenue, а потом с помощью простой линейной регрессии расчитывать EBITDA.

Посчитаем теперь и визуализируем корреляционную матрицу по всем признакам, включая наши таргет переменные. Помимо VNOM уберём из анализа ещё LONE и MGY, так как у них есть пропуски.

In [ ]:
df_corr = df_train[~(df_train['Company'].isin(['VNOM', 'LONE', 'MGY']))].groupby('Company').corr().reset_index()

In [ ]:
companies = df_corr['Company'].unique()

corr_matrix = df_corr[df_corr['Company'] == companies[0]].drop(columns='Company').set_index('level_1')
for company in companies[1:]:
    corr_ = df_corr[df_corr['Company'] == company].drop(columns='Company').set_index('level_1')
    corr_matrix = corr_matrix.add(corr_, fill_value=0)
corr_matrix = corr_matrix / len(companies)
corr_matrix.index.name=None

In [ ]:
fig = px.imshow(corr_matrix, text_auto=True, width=900, height=900)
fig.update_xaxes(side="top")
fig.show(renderer="svg")

Как мы можем видеть, у нас достаточно много сильно коррелированных переменных, что может нам помешать при построении моделей в дальнейшем.

Если рассматривать целевые переменные, то можно выделить только Cash From Financing Activities, так как он практически с ними не коррелирует. Все же остальные признаки значимо коррелирует с целевыми переменными.

# Metric

**Average SMAPE**. Считаем для каждой компании SMAPE и потом его усредняем как среднее арифметическое.
 
Одна из идей: считать SMAPE сразу по всем прогнозам и потом уже усреднять. В таком случае компании, по которым у нас меньше данных, получат меньший вес в итоговой оценке. То есть так мы покажем, что там страшнее допустить ошибку по компании, у которой больше данных, чем по компании, у которой меньше.


Ниже представлена формула SMAPE для одной компании

<img src="images/SMAPE.webp" />

Я предлагаю считать среднюю SMAPE отдельно для Revenue и отдельно для EBITDA, после чего можно просто усреднить два этих показателя. Интуитивно кажется, что они будут примерно равны, так как сильно скоррелированы между собой. Поэтому большой разницы между тремя показателями SMAPE_Revenue, SMAPE_EBITDA и SMAPE_Average быть не должно. 

# Models

В качестве бейзлайна я бы попробовал обучить модель, которая обучается только на одном временном ряду и на одной категориальной переменной, отвечающей за компанию. Для Revenue и EBITDA можно обучать две разные модели. 